In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import tqdm
import itertools
import warnings
warnings.filterwarnings("ignore")

In [2]:
moorings = ['f11','f12','f13','f14']

In [3]:
### monthly mean data at each mooring

datapath = '/Users/carmennab/Dropbox/alpha_retracker/data/sit/pysiral/'

radar_freeboards_data = {} ; aux_data = {}

for mooring in moorings:

    radar_freeboards_data[mooring] = pd.read_csv(datapath+'mean_rf_mooring_'+mooring+'.csv')
    aux_data[mooring] =  pd.read_csv('../data/sit/pysiral/aux_'+mooring+'.csv')

In [4]:
c = 3.
cs = 2.4
rho_w = 1023.9

### make ERA5 SITS, MERRA2 SITS, JUT SITS

In [33]:
alphas = np.round(np.arange(0,1.01,0.01),decimals=2)
thresholds = np.arange(5,97.5,2.5)
unique_months = set(radar_freeboards_data['f11']['month'])
unique_years = set(radar_freeboards_data['f11']['year'])

for name in ['era5','merra2','jut']:
    
    for alpha in tqdm.tqdm(alphas):

        sits = []

        for month, year in itertools.product(unique_months,unique_years):

            d = {'year':year,'month':month}
            
            my_buoy_sits = [] 
            
            for mooring in moorings:
                aux_mooring = aux_data[mooring]
                df_buoy_thickness = aux_mooring['mean_buoy_thickness_'+name]
                buoy_thickness = df_buoy_thickness[(aux_mooring['month']==month) & (aux_mooring['year']==year)]
                
                if (len(buoy_thickness)>0):
                    my_buoy_sits.append(buoy_thickness.iloc[0])
                    
            mean_buoy_sit = np.nanmean(my_buoy_sits)
            
            if ~np.isnan(mean_buoy_sit):
                    
                d['mean_buoy_thickness'] = mean_buoy_sit

                for threshold in thresholds:
                    my_thresh_sits = [] 

                    for mooring in moorings:
                        aux_mooring = aux_data[mooring]

                        if name == 'jut':
                            df_rho_i = aux_mooring['mean_ice_density_jut']
                            df_hs = aux_mooring['mean_era5_snod']
                            df_rho_s = aux_mooring['mean_era5_sden']

                        else:
                            df_rho_i = aux_mooring['mean_ice_density']
                            df_hs = aux_mooring['mean_'+name+'_snod']
                            df_rho_s = aux_mooring['mean_'+name+'_sden']

                        rfs = radar_freeboards_data[mooring]

                        freeboard = rfs[(rfs['month']==month) & (rfs['year']==year)]

                        if len(freeboard)>0:

                            rf = rfs['mean_radar_freeboard'][(rfs['month']==month) & (rfs['year']==year) & (rfs['threshold']==threshold)].iloc[0]

                            hs = df_hs[(aux_mooring['month']==month) & (aux_mooring['year']==year)].iloc[0]

                            rho_s = df_rho_s[(aux_mooring['month']==month) & (aux_mooring['year']==year)].iloc[0]
                            rho_i = df_rho_i[(aux_mooring['month']==month) & (aux_mooring['year']==year)].iloc[0]
                            ice_freeboard = rf + (((alpha * (c/cs)) -1)*hs)  # isobel thesis pg 114
                            sit = ((ice_freeboard*rho_w) + (hs*rho_s)) / (rho_w - rho_i)

                            my_thresh_sits.append(sit)

                    d['mean_sit_'+str(threshold)] = np.nanmean(my_thresh_sits)

                sits.append(d)

            sit_df = pd.DataFrame(sits)
            filepath = f'../data/sit/pysiral/alpha_sits_{name}/a{str(alpha)}_sumata.csv'
            sit_df.to_csv(filepath, na_rep=np.nan, index=False)

100%|█████████████████████████████████████████| 101/101 [11:46<00:00,  7.00s/it]


### make ERA5 * 0.5 and * 1.5 SITS

In [5]:
alphas = np.round(np.arange(0,1.01,0.01),decimals=2)
thresholds = np.arange(5,97.5,2.5)
unique_months = set(radar_freeboards_data['f11']['month'])
unique_years = set(radar_freeboards_data['f11']['year'])

for name in ['90','110']:
    
    for alpha in tqdm.tqdm(alphas):

        sits = []

        for month, year in itertools.product(unique_months,unique_years):

            d = {'year':year,'month':month}
            
            my_buoy_sits = [] 
            
            for mooring in moorings:
                aux_mooring = aux_data[mooring]
                df_buoy_thickness = aux_mooring['mean_buoy_thickness_'+name]
                buoy_thickness = df_buoy_thickness[(aux_mooring['month']==month) & (aux_mooring['year']==year)]
                
                if (len(buoy_thickness)>0):
                    my_buoy_sits.append(buoy_thickness.iloc[0])
                    
            mean_buoy_sit = np.nanmean(my_buoy_sits)
            
            if ~np.isnan(mean_buoy_sit):
                    
                d['mean_buoy_thickness'] = mean_buoy_sit

                for threshold in thresholds:
                    my_thresh_sits = [] 

                    for mooring in moorings:
                        aux_mooring = aux_data[mooring]

                        df_rho_i = aux_mooring['mean_ice_density']
                        df_hs = aux_mooring['mean_era5_snod_'+name]
                        df_rho_s = aux_mooring['mean_era5_sden']

                        rfs = radar_freeboards_data[mooring]

                        freeboard = rfs[(rfs['month']==month) & (rfs['year']==year)]

                        if len(freeboard)>0:

                            rf = rfs['mean_radar_freeboard'][(rfs['month']==month) & (rfs['year']==year) & (rfs['threshold']==threshold)].iloc[0]

                            hs = df_hs[(aux_mooring['month']==month) & (aux_mooring['year']==year)].iloc[0]

                            rho_s = df_rho_s[(aux_mooring['month']==month) & (aux_mooring['year']==year)].iloc[0]
                            rho_i = df_rho_i[(aux_mooring['month']==month) & (aux_mooring['year']==year)].iloc[0]
                            ice_freeboard = rf + (((alpha * (c/cs)) -1)*hs)  # isobel thesis pg 114
                            sit = ((ice_freeboard*rho_w) + (hs*rho_s)) / (rho_w - rho_i)

                            my_thresh_sits.append(sit)

                    d['mean_sit_'+str(threshold)] = np.nanmean(my_thresh_sits)

                sits.append(d)

            sit_df = pd.DataFrame(sits)
            filepath = f'../data/sit/pysiral/alpha_sits_{name}/a{str(alpha)}_sumata.csv'
            sit_df.to_csv(filepath, na_rep=np.nan, index=False)

100%|█████████████████████████████████████████| 101/101 [12:17<00:00,  7.30s/it]
